# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

In [1]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [2]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [4]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [5]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31314    0 31314    0     0  43416      0 --:--:-- --:--:-- --:--:-- 43371


In [6]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70173    0 70173    0     0   281k      0 --:--:-- --:--:-- --:--:--  280k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [7]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

[nltk_data] Downloading package punkt to /home/nageshbm/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/nageshbm/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [8]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/mnt/c/Users/email/Desktop/bootcamp/AIE5/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/mnt/c/Users/email/Desktop/bootcamp/AIE5/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/mnt/c/Users/email/Desktop/bootcamp/AIE5/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [9]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [10]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [11]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 14, relationships: 67)

We can save and load our knowledge graphs as follows.

In [12]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 14, relationships: 67)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [13]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [14]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


#### ✅ Answer:

The 3 query synthesizers functions are:

1. SingleHopSpecificQuerySynthesizer: This synthesizer generates questions that are specific to a single document.
2. MultiHopAbstractQuerySynthesizer: This synthesizer generates questions that are abstract and require multiple hops to answer.
3. MultiHopSpecificQuerySynthesizer: This synthesizer generates questions that are specific to a single document but require multiple hops to answer.



Finally, we can use our `TestSetGenerator` to generate our testset!

In [15]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What contributions has Microsoft Research made...,[Code may be the best application The ethics o...,Microsoft Research has contributed to the deve...,single_hop_specifc_query_synthesizer
1,What was Andy Baio's involvement in the story ...,[Based Development As a computer scientist and...,"Andy Baio, along with the author, produced the...",single_hop_specifc_query_synthesizer
2,What is the significance of the 1950s in the c...,[Simon Willison’s Weblog Subscribe Stuff we fi...,The 1950s mark the beginning of the academic f...,single_hop_specifc_query_synthesizer
3,What role does Stanford Alpaca play in the dev...,[easy to follow. The rest of the document incl...,Stanford Alpaca is mentioned in the context of...,single_hop_specifc_query_synthesizer
4,What advancements have been made with Claude 3...,[Prompt driven app generation is a commodity a...,"Claude 3.5 Sonnet, launched in June, became a ...",single_hop_specifc_query_synthesizer
5,How have technological advancements in 2024 in...,[<1-hop>\n\nPrompt driven app generation is a ...,"In 2024, technological advancements significan...",multi_hop_abstract_query_synthesizer
6,How have technological advancements in 2024 co...,[<1-hop>\n\nPrompt driven app generation is a ...,"In 2024, technological advancements significan...",multi_hop_abstract_query_synthesizer
7,How have advancements in large language models...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in large language models (LLMs) h...,multi_hop_abstract_query_synthesizer
8,How did the advancements in GPT-4 and Claude A...,[<1-hop>\n\nneeds guidance. Those of us who un...,"In 2024, advancements in GPT-4 and Claude Arti...",multi_hop_specific_query_synthesizer
9,How have the developments in Llama 2 and Llama...,[<1-hop>\n\neasy to follow. The rest of the do...,The developments in Llama 2 and Llama 3 have b...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [16]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [17]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,what happen with LLMs in 2023 and why they imp...,[Code may be the best application The ethics o...,"In 2023, significant developments occurred in ...",single_hop_specifc_query_synthesizer
1,What are the primary challenges associated wit...,[Based Development As a computer scientist and...,The primary challenges associated with LLMs fr...,single_hop_specifc_query_synthesizer
2,What significant developments regarding Large ...,[Simon Willison’s Weblog Subscribe Stuff we fi...,2023 was the breakthrough year for Large Langu...,single_hop_specifc_query_synthesizer
3,Wht r the ethical implcations of using GPT-4V ...,[easy to follow. The rest of the document incl...,The ethical implications of using GPT-4V in AI...,single_hop_specifc_query_synthesizer
4,How does the use of synthetic training data co...,[<1-hop>\n\nPrompt driven app generation is a ...,The use of synthetic training data plays a sig...,multi_hop_abstract_query_synthesizer
5,How have efficiency improvements in LLMs impac...,[<1-hop>\n\nPrompt driven app generation is a ...,Efficiency improvements in LLMs have significa...,multi_hop_abstract_query_synthesizer
6,How have the advancements in LLM efficiency an...,[<1-hop>\n\nPrompt driven app generation is a ...,The advancements in LLM efficiency and pricing...,multi_hop_abstract_query_synthesizer
7,How does the use of synthetic training data co...,[<1-hop>\n\nPrompt driven app generation is a ...,The use of synthetic training data plays a sig...,multi_hop_abstract_query_synthesizer
8,How does the training cost and environmental i...,[<1-hop>\n\nday after that. DeepSeek v3 is a h...,"DeepSeek v3, a 685B parameter model, was train...",multi_hop_specific_query_synthesizer
9,How has the development of GPT-4o and other mu...,[<1-hop>\n\ngets you OpenAI’s most expensive m...,The development of GPT-4o and other multi-moda...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [18]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [19]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [20]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [22]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [23]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

In [24]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [25]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [26]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Finally, we can set-up our RAG LCEL chain!

In [27]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [28]:
rag_chain.invoke({"question" : "What are Agents?"})

'"Agents" are referred to as AI systems that can act on behalf of a user, but the term lacks a single, clear definition. Some people think of agents as similar to travel agents, while others see them as large language models (LLMs) that can run tools to solve problems. The concept is still considered vague and has not yet materialized into fully functional systems in production, despite the excitement around them. There is also skepticism about their utility due to challenges related to gullibility; LLMs tend to believe anything they are told, which can limit their effectiveness as decision-makers.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4o as our evaluation LLM for our base Evaluators.

In [29]:
eval_llm = ChatOpenAI(model="gpt-4o")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [30]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dope_or_nope_evaluator`:

## LangSmith Evaluation

In [31]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'crushing-insect-92' at:
https://smith.langchain.com/o/5893d499-6998-4f44-a84d-2fcf6d99ac9b/datasets/f3257b9d-6805-4455-a746-2ef10159f438/compare?selectedSessions=18d9a308-819a-42c6-a8ac-efe896547e1f




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,"In 2023, what were the ethical challenges asso...","In 2023, the ethics of Large Language Models (...",None,"In 2023, the ethical challenges associated wit...",1,1,0,2.413540,79383b6b-f9a6-4444-8201-e8b813c02bcb,310f9768-f031-4c05-abeb-974792a95bb1
1,What are the ethical considerations and challe...,I don't know.,None,The ethical considerations and challenges asso...,0,0,0,1.584288,b6b6d828-1ce6-426d-92c1-7132333e1f1d,69f697f1-520d-40c4-a182-e8b07cadc5fa
2,How has the development of GPT-4o and other mu...,I don't know.,None,The development of GPT-4o and other multi-moda...,0,0,0,1.039013,962e048d-8072-43a2-bcab-e72371735f4a,228ea918-8074-4353-a39f-16a7281f1438
3,How does the training cost and environmental i...,The training cost of DeepSeek v3 was approxima...,None,"DeepSeek v3, a 685B parameter model, was train...",1,0,1,9.072412,10b8d124-60e9-45c8-95f8-aad8f165c75e,274b4451-b698-49a5-a777-00a73c06856f
4,How does the use of synthetic training data co...,The use of synthetic training data contributes...,None,The use of synthetic training data plays a sig...,1,1,0,4.303799,8ae498ab-0f6b-4cea-8b91-d55d3f8bacc7,8dad82be-74ef-4442-8ce3-67a86b23fb11
5,How have the advancements in LLM efficiency an...,The advancements in LLM efficiency and the res...,None,The advancements in LLM efficiency and pricing...,1,0,0,2.201940,b7ca63a0-a0bf-4682-bf3f-d336c6815e10,60ee8b5d-5a37-4504-9dc9-2f008cc8ca12
6,How have efficiency improvements in LLMs impac...,Efficiency improvements in LLMs have led to a ...,None,Efficiency improvements in LLMs have significa...,1,1,0,2.214681,f0179813-8098-4a16-9f7a-9f8331e29874,4ced4f27-2517-4295-87bc-480f9794e74e
7,How does the use of synthetic training data co...,The use of synthetic training data contributes...,None,The use of synthetic training data plays a sig...,1,1,0,3.285569,15486d29-1c1b-4fa4-af89-0a8edc495593,15823b6c-96b4-4b65-87ed-71a6feed3471
8,Wht r the ethical implcations of using GPT-4V ...,The ethical implications of using GPT-4 and si...,None,The ethical implications of using GPT-4V in AI...,1,1,0,2.354107,00bfb038-9320-47fc-ad27-49bf3414251b,c588f43c-ee13-4f8c-8748-7f90e0056ffc
9,What significant developments regarding Large ...,"In 2023, significant developments regarding La...",None,2023 was the breakthrough year for Large Langu...,1,0,0,2.907230,eec0f137-db3a-4faf-942c-2bf8e065d842,d777a654-bc96-4bd3-8496-a2bc11c408aa


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [32]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [33]:
rag_documents = docs

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

#### ✅ Answer:

Modifying the chunk size can affect the performance of our application in the following ways:

1. **Contextual Accuracy**: A larger chunk size allows the model to consider more of the relevant context, potentially leading to more accurate answers.

2. **Memory Usage**: Larger chunks require more memory, which can impact the performance of the application.

3. **Speed**: Larger chunks can slow down the generation process, as the model has to process more text.

4. **Quality**: A larger chunk size can lead to a loss of important details, which can impact the quality of the answers.

5. **Relevance**: A larger chunk size can lead to a loss of important details, which can impact the quality of the answers.

In [35]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

#### ✅ Answer:

Modifying the embedding model can affect the performance of our application in the following ways:

1. **Contextual Accuracy**: A larger chunk size allows the model to consider more of the relevant context, potentially leading to more accurate answers.

2. **Memory Usage**: Larger chunks require more memory, which can impact the performance of the application.

3. **Speed**: Larger chunks can slow down the generation process, as the model has to process more text.

4. **Quality**: A larger chunk size can lead to a loss of important details, which can impact the quality of the answers.

5. **Relevance**: A larger chunk size can lead to a loss of important details, which can impact the quality of the answers.

In [36]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [37]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [38]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [39]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

'Agents are these elusive AI concepts that people casually throw around, but the term is super vague. It usually refers to AI systems that are meant to act on your behalf, like a digital assistant or a travel agent. However, there’s a lot of confusion about what "agents" really mean, with various definitions and interpretations flying around. Some think of them as LLMs (Large Language Models) with tool access, while others have no clear picture at all. Basically, everyone’s excited about the potential of agents, but they still feel like they’re "coming soon" without concrete examples running in the real world. And the whole thing gets complicated by issues like gullibility—making it hard for these systems to distinguish truth from fiction. So yeah, cool idea, but still kinda fuzzy!'

Finally, we can evaluate the new chain on the same test set!

In [40]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'yellow-vase-78' at:
https://smith.langchain.com/o/5893d499-6998-4f44-a84d-2fcf6d99ac9b/datasets/f3257b9d-6805-4455-a746-2ef10159f438/compare?selectedSessions=bc74b64e-e302-4cba-b6fd-42c2f14b7a4a




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,"In 2023, what were the ethical challenges asso...","Hey there! So in 2023, the wild world of Large...",None,"In 2023, the ethical challenges associated wit...",1,0,1,2.741319,79383b6b-f9a6-4444-8201-e8b813c02bcb,cc21f16e-0549-493b-82fb-c6fdd5a0281e
1,What are the ethical considerations and challe...,"Yo, so here’s the deal. When it comes to train...",None,The ethical considerations and challenges asso...,1,0,1,3.696941,b6b6d828-1ce6-426d-92c1-7132333e1f1d,09aa64c5-a22d-4c1e-997a-3a95d9e8bba8
2,How has the development of GPT-4o and other mu...,"Yo, let's break it down! The development of GP...",None,The development of GPT-4o and other multi-moda...,1,0,1,2.905368,962e048d-8072-43a2-bcab-e72371735f4a,8afb2175-a459-4e98-9d31-399cf2a16bc6
3,How does the training cost and environmental i...,"Yo, check it out! DeepSeek v3 came in hot with...",None,"DeepSeek v3, a 685B parameter model, was train...",1,1,1,5.011390,10b8d124-60e9-45c8-95f8-aad8f165c75e,c6fd011a-4f19-46ad-a07b-8039f533bdb9
4,How does the use of synthetic training data co...,"Alright, let's break it down! The use of synth...",None,The use of synthetic training data plays a sig...,1,0,1,5.164906,8ae498ab-0f6b-4cea-8b91-d55d3f8bacc7,d41e3f42-ae10-48a1-9e77-aaad8043bb7a
5,How have the advancements in LLM efficiency an...,"Yo, check this out! The advancements in LLM ef...",None,The advancements in LLM efficiency and pricing...,1,0,1,4.344474,b7ca63a0-a0bf-4682-bf3f-d336c6815e10,29c740b5-a7b0-44ee-8ac1-4a9818bb7fde
6,How have efficiency improvements in LLMs impac...,"Yo, check it out! The efficiency improvements ...",None,Efficiency improvements in LLMs have significa...,1,0,1,3.597810,f0179813-8098-4a16-9f7a-9f8331e29874,4b08b106-a117-4143-aca9-35ad56b81671
7,How does the use of synthetic training data co...,Synthetic training data is like the secret sau...,None,The use of synthetic training data plays a sig...,1,0,1,3.253192,15486d29-1c1b-4fa4-af89-0a8edc495593,d0375c7d-a27c-470f-b2cd-1e8b4bbbc921
8,Wht r the ethical implcations of using GPT-4V ...,"Yo, the ethical implications of using models l...",None,The ethical implications of using GPT-4V in AI...,1,1,1,3.787626,00bfb038-9320-47fc-ad27-49bf3414251b,4beab648-d31f-4dbc-9309-f80d3855f9fe
9,What significant developments regarding Large ...,"In 2023, it was a major breakthrough year for ...",None,2023 was the breakthrough year for Large Langu...,1,1,1,4.441013,eec0f137-db3a-4faf-942c-2bf8e065d842,87ec126d-38bd-46bb-bb94-f9e0a3d3bef7


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

#### ✅ Answer:

- `qa_evaluator`: This evaluator checks if the generated answer is correct and matches the reference answer.
- `labeled_helpfulness_evaluator`: This evaluator checks if the generated answer is helpful to the user by considering the correct reference answer.
- `dope_or_nope_evaluator`: This evaluator checks if the generated answer is dope, lit, or cool.


#### ✅ Answer:

Evaluation 1 screenshot:

![alt text](<Screenshot 2025-02-08 140909.png>)

Evaluation 2 screenshot:

![alt text](<Screenshot 2025-02-08 140929.png>)



Analysis of Evaluation 1 and Evaluation 2:
Summary of Differences

1. Correctness:
Evaluation 1: There are several instances where the model failed to provide correct answers (e.g., questions 1, 2, 10, and 11).
Evaluation 2: The model improved significantly, providing correct answers for all but one question (question 11).

2. Helpfulness:
Evaluation 1: Helpfulness scores are low for many responses, indicating that even when answers were provided, they were not deemed helpful (e.g., questions 1, 2, 5, 10, and 11).
Evaluation 2: Helpfulness scores improved, with many responses rated as helpful, particularly for questions 0, 3, 4, 5, 6, 7, 8, 9, and 10.

3. Dopeness:
Evaluation 1: The dopeness score is low for many responses, indicating a lack of engaging or creative responses (e.g., questions 1, 2, 5, 10, and 11).
Evaluation 2: The dopeness score improved, with many responses rated as "dope" or engaging, particularly for questions 0, 3, 4, 5, 6, 7, 8, 9, and 10.

4. Execution Time:
Evaluation 1: Execution times vary, with some responses taking longer than others (e.g., question 3 took 9.072 seconds).
Evaluation 2: Execution times are generally lower, indicating improved efficiency in generating responses (e.g., question 3 took 5.011 seconds).
Explanation of Changes
Improved Model Performance:
The transition from Evaluation 1 to Evaluation 2 involved modifications to the RAG chain, including the use of a "dope" prompt and a larger embedding model. These changes likely contributed to the model's ability to generate more accurate, helpful, and engaging responses.
Prompt Engineering:
The introduction of a more conversational and engaging prompt in Evaluation 2 likely encouraged the model to produce responses that were not only correct but also more relatable and interesting. This is evident in the increased dopeness scores.
Increased Contextual Awareness:
The adjustments made to the chunk size and embedding model may have enhanced the model's contextual understanding, allowing it to provide more relevant and accurate answers. This is reflected in the higher correctness and helpfulness scores.

5. Feedback Loop:
The evaluation process itself may have provided insights that informed further refinements to the model. The feedback from Evaluation 1 could have highlighted areas for improvement, leading to the enhancements seen in Evaluation 2.
Conclusion
Overall, the changes made between Evaluation 1 and Evaluation 2 resulted in a significant improvement in the model's performance across various metrics. The combination of better prompt design, enhanced model capabilities, and a focus on engaging responses contributed to the observed improvements in correctness, helpfulness, dopeness, and execution time.